In [4]:
# import libraries and modules

import time
import numpy as np
import h5py
import math
import astropy_healpix as ah
from astropy import units as u
import healpy as hp
import ligo.skymap.plot           
from ligo.skymap.tool import ArgumentParser, FileType
from ligo.skymap.io import fits
from ligo.skymap.tool import ligo_skymap_contour
from ligo.skymap.postprocess import contour
from ligo.skymap import postprocess

import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.time import Time

import scipy.stats as st
from scipy.optimize import bisect

from ligo.skymap import io
from ligo.skymap.bayestar import rasterize
from ligo.skymap import version
from astropy.table import Table
from astropy.time import Time
import numpy as np
import os
import sys
import pickle
from ligo.skymap.kde import Clustered2Plus1DSkyKDE, Clustered2DSkyKDE
import logging
from textwrap import wrap


In [14]:
ifos = ['L1','H1', 'V1', 'K1']

name = "".join(ifos)

total_injections = 400

count = 0
# for count in range(1):
    
with h5py.File(os.getcwd() + '/post_samples_interp_SNR_20to25_{}.hdf'.format(name), 'a') as file :
    
    group = file['injection_{}'.format(count)]
    #print(group.keys())
    ra_samps = np.array(group['ra'])
    dec_samps = np.array(group['dec'])

In [12]:
ra_samps

array([1.68975563, 1.6903174 , 1.71794693, ..., 1.70428846, 1.70436459,
       1.65694205])

In [18]:
pts = np.column_stack((ra_samps, dec_samps))
pts.shape

(3102, 2)

In [22]:
del_omega = np.zeros(total_injections)
del_omega.shape

(400,)

In [20]:
trials = 1
jobs = 1

skypost = Clustered2DSkyKDE(pts, trials=trials, jobs=trials)

hpmap = skypost.as_healpix(top_nside=16)

io.write_sky_map(os.getcwd() + '/fits_files/injection_{}_skymap.fits.gz'.format(count), hpmap, nest=True)

#-- main code --

skymap, metadata = fits.read_sky_map(os.getcwd() + '/fits_files/injection_{}_skymap.fits.gz'.format(count), nest=None)
nside = ah.npix_to_nside(len(skymap))

# Convert sky map from probability to probability per square degree.

deg2perpix = ah.nside_to_pixel_area(nside).to_value(u.deg**2)
probperdeg = skymap / deg2perpix

levels = [90]   # this should be provided as a list
                #(see: https://github.com/lpsinger/ligo.skymap/blob/main/ligo/skymap/tool/ligo_skymap_plot.py)
    
# for contours 

vals = 100 * postprocess.find_greedy_credible_levels(skymap)

# for area of 90% credible region 

pp = np.round([90]).astype(int)               
area = (np.searchsorted(np.sort(vals), [90]) * deg2perpix)[0]
      
print('injection {}, area: {}'.format(count, area))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:57<00:00,  1.44s/it]


adaptive refinement round 1 of 7 ...
adaptive refinement round 2 of 7 ...
adaptive refinement round 3 of 7 ...
adaptive refinement round 4 of 7 ...
adaptive refinement round 5 of 7 ...
adaptive refinement round 6 of 7 ...
adaptive refinement round 7 of 7 ...
injection 0, area: 1.9998793889797333


In [25]:
with h5py.File(os.getcwd() + '/quantities_of_interest.hdf', 'r') as f:
    
    group = f['L1H1V1K1']
    
    delta_omega = np.array(group['delta_omega'])

In [26]:
delta_omega

array([1.99987939, 2.98998361])